Remember to 'pip install [package_name]' if these imports are not found

In [ ]:
# Imports from selenium, request_futures, requests, bs4, pandas
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from requests_futures.sessions import FuturesSession
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

# Native python imports
import re
import math
from collections import Counter
from concurrent.futures import as_completed

# Enter Lat /Lon coordinates and Day of Interest
These should be the only inputs needed before executing the rest of the script.

In [ ]:
# Defining constants for our current usecase
day_of_interest = 51 # Day of Year (DOY) of February 20, which is our current examination
g_truth_lat, g_truth_lon = 75.92, -149.69 # Selecting a coordinate of interest (EX: Center of SAR image)

### Run this section to get references to the buoy data
The buoy data is listed as a table of ids that is nested within an iframe of the website. Because of this iframe, we use selenium to simulate a web-client until the data loads, then grab it using BeautifulSoup.

In [ ]:
#####
base_url = f"https://iabp.apl.uw.edu/IABP_Table.html"

options = webdriver.ChromeOptions()
options.add_argument('--headless')
# executable_path param is not needed if you updated PATH
browser = webdriver.Chrome(options=options)

try:
    browser.get(base_url)
    timeout_in_seconds = 10
    WebDriverWait(browser, timeout_in_seconds).until(ec.frame_to_be_available_and_switch_to_it("myframe"))
    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")
    print("Got the site!")
except TimeoutException:
    print("I give up...")
finally:
    browser.quit()
    
table = soup.find('table')
tablebody = soup.find('tbody')
tablerow = tablebody.find_all('tr')

buoy_ids = []
failed = 0
for row in tablerow:
    buoy_id = row.find('td')
    try:
        buoy_ids.append(buoy_id.get_text())
    except:
        failed += 1
num_buoys = len(buoy_ids)
print(f"Buoys to search through: {num_buoys}")
print(f"Rows without buoy ids: {failed}")

### Run this next section to populate the buoy dictionary with distances
Get all of the data from the different buoys, filter to only those that have record of our time of interest, and then calculate their distances from our point of interest.

In [ ]:
# Defining url getter
def get_url(url:str):
    return f"https://iabp.apl.uw.edu/WebData/{url}.dat"

regExStr = f'\d{{5,20}}\s*?2023(?:\s*?\d{{2}}){{2}}\s*?(?:{day_of_interest}|{day_of_interest + 1}).*'

# Declare our dictionary and counters
distances_dict = {}
num_completed = 0

# Open a session to speed up the time it takes to make and receive all of our requests
with FuturesSession() as session:
    
    # Get the url for each buoy id, and then get the data
    urls = [get_url(id_num) for id_num in buoy_ids]
    futures = [session.get(url) for url in urls]
    
    # Event listener to execute as the requests resolve
    for future in as_completed(futures):
        print(f"{num_completed + 1} / {num_buoys} requests resolved")
        
        # Convert from byte to string format
        response = future.result().content.decode('UTF-8')
        
        # Get the column headers (and removing whitespace) using the split method
        column_headers = response.partition('\n')[0].split()
        
        # Find all rows of buoy data using regex
        daily_data = re.findall(regExStr, response)
        daily_data_len = len(daily_data)
        
        # Iterate through the data per buoy to calculate distance
        if daily_data_len > 0:
            buoy_id = daily_data[0].split()[0]
            buoy_dist = 0
            print(f"Processing {num_completed+1}/{num_buoys} pages, with {daily_data_len} rows.")
            for row in daily_data:
                try:
                    # Calculating avg distance of buoy 
                    lat_index = column_headers.index("Lat")
                    lon_index = column_headers.index("Lon")
                    splits = row.split()
                    lat = splits[lat_index]
                    lon = splits[lon_index]
                    seg_dist = math.dist([float(lat), float(lon)], [g_truth_lat, g_truth_lon])
                    buoy_dist += seg_dist
                except:
                    print("Could not find lat/lon column for ", buoy_id)
                    
            # Set the distance in the dictionary
            avg_dist = buoy_dist / daily_data_len
            distances_dict[buoy_id] = avg_dist
            print(f"Finished processing {buoy_id}. Average Dist = {avg_dist}")        
        else:
            print(f"Request {num_completed + 1} has no matching data.", )
        num_completed += 1
        

### Run this section to extract the data from the nearest buoys

In [ ]:
fileName = "Nearest5Buoys.csv"
num_buoys = 5

df_cols = []
closest_buoys = list(Counter(distances_dict).most_common()[-num_buoys:])
closest_buoys.reverse()
for b_id, dist in closest_buoys:

    data_frame_data = []
    response = get(get_url(b_id)).content.decode('UTF-8')
    daily_data = re.findall(regExStr, response)
    if len(daily_data) > 0:
        column_headers = response.partition('\n')[0].split()
        # Make sure we have all column headers
        for header in column_headers:
            if header not in df_cols:
                df_cols.append(header)
        # Then push in the data so we can later create the df
        for i, row in enumerate(daily_data):
            splits = row.split()
            data_frame_data.append(splits)

# If the columns don't match up, we have pulled data from the optional columns and need to make sure they're in our headers
data_csv = pd.DataFrame(data_frame_data, columns=column_headers)
print(data_csv.head(5))
print(data_csv.tail(5))

# Comment / Uncomment this last line to toggle whether the file gets written
data_csv.to_csv(fileName)